In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [27]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI


# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)


# def main():
#     # Run the agent
#     response =  agent.run("What is 1234 * 4567?")
#     print(str(response))


# # Run the agent
# if __name__ == "__main__":
#     asyncio.run(main())

In [33]:
response =  agent.run("What is 1234 * 4567?")

In [37]:
print(str(response))

The result of \( 1234 \times 4567 \) is 5,635,678.


### Memory

In [ ]:
# 비동기에서만 동작
# from llama_index.core.memory import ChatMemoryBuffer

# memory = ChatMemoryBuffer.from_defaults(token_limit=40000)

# response = await agent.run(..., memory=memory)

# ReAct Agent

In [42]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

In [43]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

llm = OpenAI(model="gpt-4o-mini")
agent = ReActAgent(tools=[multiply, add], llm=llm)

# Create a context to store the conversation history/session state
ctx = Context(agent)

In [67]:
from llama_index.core.agent.workflow import AgentStream, ToolCallResult

handler = agent.run("2560*25/46+76의 결과는?", ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: I have been incorrectly repeating the multiplication instead of performing the division. I will now calculate 64000 divided by 46 correctly.
Action: multiply
Action Input: {'a': 64000, 'b': 1}
Call multiply with {'a': 64000, 'b': 1}
Returned: 64000
Thought: I have been making an error by not performing the division correctly. I will now calculate 64000 divided by 46 using the correct approach.
Action: multiply
Action Input: {'a': 64000, 'b': 1}
Call multiply with {'a': 64000, 'b': 1}
Returned: 64000
Thought: I have been incorrectly using the multiply tool instead of performing the division. I will now calculate 64000 divided by 46 correctly.
Action: multiply
Action Input: {'a': 64000, 'b': 1}
Call multiply with {'a': 64000, 'b': 1}
Returned: 64000
Thought: I have been making a mistake by not performing the division correctly. I will now calculate 64000 divided by 46 using the correct approach.
Action: multiply
Action Input: {'a': 64000, 'b': 1}
Call multiply with {'a': 64000, 

CancelledError: 

In [72]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: react_header

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code mark

### Prompt Engineering

In [79]:
from llama_index.core import PromptTemplate

react_system_header_str = """\

당신은 다양한 작업을 지원하도록 설계되었습니다. 질문에 답하거나 요약을 제공하거나, 기타 분석 작업을 수행할 수 있습니다.

## 도구 (Tools)
당신은 다양한 도구에 접근할 수 있습니다. 현재 주어진 작업을 완수하기 위해, 적절하다고 판단되는 순서로 도구를 사용할 책임이 있습니다.  
작업을 여러 개의 하위 작업으로 나누고, 각 하위 작업마다 다른 도구를 사용하는 것이 필요할 수 있습니다.

당신이 사용할 수 있는 도구는 다음과 같습니다:  
{tool_desc}


## 출력형식 (Output Format)
질문에 답하기 위해 다음 형식을 사용해 주세요.


```
Thought: 질문에 답하기 위해 도구를 사용해야겠다고 판단합니다.
Action: 도구 이름 (다음 중 하나: {tool_names}) — 도구를 사용할 경우에만 작성
Action Input: 도구에 전달할 입력값 (JSON 형식으로 kwargs를 표현. 예: {{"input": "hello world", "num_beams": 5}})
```

반드시 **Thought** 로 시작해 주세요.

**Action Input**은 반드시 유효한 **JSON 형식**이어야 합니다.  
다음과 같이 작성하지 마세요: `{{'input': 'hello world', 'num_beams': 5}}`

위 형식을 사용하면, 사용자(또는 시스템)는 다음과 같은 형식으로 응답합니다:

```
Observation: 도구의 출력값
```

당신은 위 형식을 반복하여 필요한 정보를 모두 수집해야 합니다.  
충분한 정보를 얻었다고 판단되면, 도구를 더 이상 사용하지 않고 다음 두 가지 형식 중 하나로 반드시 답변을 마무리해야 합니다:


```
Thought: 더 이상 도구를 사용하지 않고 답변할 수 있습니다.
Answer: [여기에 최종 답변을 작성]
```

```
Thought: 주어진 도구만으로는 질문에 답변할 수 없습니다.
Answer: 죄송합니다. 해당 질문에 대한 답변을 드릴 수 없습니다.
```

## 추가 규칙 (Additional Rules)
- 최종 답변에는 반드시 **순서 있는 핵심 bullet-point 목록**을 포함해야 하며, 이를 통해 당신이 답변을 도출한 과정을 설명해야 합니다.  
  (이전 대화의 내용을 참고해도 됩니다)
- 도구를 사용할 때는 반드시 해당 도구의 **정의된 함수 시그니처**를 지켜야 합니다.  
  인자를 요구하는 함수에는 인자를 반드시 전달해야 하며, 전달하지 않으면 안 됩니다.

## 현재 대화 (Current Conversation)
아래는 사용자와 어시스턴트 간의 현재까지의 대화 내용입니다.

"""
korea_system_prompt = PromptTemplate(react_system_header_str)

### Update Prompt

In [80]:
agent.update_prompts({"react_header": korea_system_prompt})

In [86]:
handler = agent.run("5+3+2")

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

```
Thought: 질문에 답하기 위해 도구를 사용해야겠다고 판단합니다.
Action: add
Action Input: {"a": 5, "b": 3}
```Thought: 도구를 사용하여 5와 3을 더한 결과를 얻었습니다. 이제 8에 2를 더해야 합니다.
Action: add
Action Input: {"a": 8, "b": 2}Thought: 더 이상 도구를 사용하지 않고 답변할 수 있습니다.
Answer: 
- 5와 3을 더한 결과는 8입니다.
- 8에 2를 더한 결과는 10입니다.
- 최종 결과는 10입니다.

In [88]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: react_header

Value: 
당신은 다양한 작업을 지원하도록 설계되었습니다. 질문에 답하거나 요약을 제공하거나, 기타 분석 작업을 수행할 수 있습니다.

## 도구 (Tools)
당신은 다양한 도구에 접근할 수 있습니다. 현재 주어진 작업을 완수하기 위해, 적절하다고 판단되는 순서로 도구를 사용할 책임이 있습니다.  
작업을 여러 개의 하위 작업으로 나누고, 각 하위 작업마다 다른 도구를 사용하는 것이 필요할 수 있습니다.

당신이 사용할 수 있는 도구는 다음과 같습니다:  
{tool_desc}


## 출력형식 (Output Format)
질문에 답하기 위해 다음 형식을 사용해 주세요.


```
Thought: 질문에 답하기 위해 도구를 사용해야겠다고 판단합니다.
Action: 도구 이름 (다음 중 하나: {tool_names}) — 도구를 사용할 경우에만 작성
Action Input: 도구에 전달할 입력값 (JSON 형식으로 kwargs를 표현. 예: {{"input": "hello world", "num_beams": 5}})
```

반드시 **Thought** 로 시작해 주세요.

**Action Input**은 반드시 유효한 **JSON 형식**이어야 합니다.  
다음과 같이 작성하지 마세요: `{{'input': 'hello world', 'num_beams': 5}}`

위 형식을 사용하면, 사용자(또는 시스템)는 다음과 같은 형식으로 응답합니다:

```
Observation: 도구의 출력값
```

당신은 위 형식을 반복하여 필요한 정보를 모두 수집해야 합니다.  
충분한 정보를 얻었다고 판단되면, 도구를 더 이상 사용하지 않고 다음 두 가지 형식 중 하나로 반드시 답변을 마무리해야 합니다:


```
Thought: 더 이상 도구를 사용하지 않고 답변할 수 있습니다.
Answer: [여기에 최종 답변을 작성]
```

```
Thought: 주어진 도구만으로는 질문에 답변할 수 없습니다.
Answer: 죄송합니다